# Joins and set operations

In this exercise, we will use the skills we have learned so far to solve complex problems that involve string manipulation and handling.



First, let's load our sample database:

In [ ]:
# Load and activate the SQL extension to allow us to execute SQL in a Jupyter notebook.
%load_ext sql

In [ ]:
# Load the Northwind database stored in your local machine. 
# Make sure the file is saved in the same folder as this notebook.
%sql sqlite:///Northwind.db

Here is a view of all of our tables in the database:

<div align="center" style=" font-size: 80%; text-align: center; margin: 0 auto">
<img src="https://raw.githubusercontent.com/Explore-AI/Pictures/master/Northwind_ERD.png"  style="width:70%";/>
<br>
<br>
    <em>Figure 1: Northwind database ERD</em>
</div>

## Overview
Run the necessary queries that will provide us with the following information. Compare your queries with the solutions at the end of this notebook.


### Exercise 1

Using the Northwind database, determine the list of products and their respective suppliers. 

### Exercise 2

Generate a list of all cities where customers and employees are located.

###  Exercise 3

Create a list of all unique cities where customers are located and employees live. 

### 4. Challenge question

Using the Northwind database, as the new sales analyst, you've been asked to prepare a sales report for the last quarter. Your report should include:
1. Customer names and their countries.
2. The total amount spent by each customer.
3. The list of products each customer purchased.
4. The total quantity of each product purchased by each customer.

Only include customers who have spent more than 5000 in the last quarter. 

## Solutions

### Exercise 1

This query combines rows from `Products` and `Suppliers` tables based on the condition where `Products.SupplierID` equals `Suppliers.SupplierID`.

Expected outcome: The result should be a table that includes columns for `ProductName` and `SupplierName` which shows the products and their respective suppliers.

In [ ]:
%%sql

SELECT 
    product.ProductName, 
    supplier.CompanyName
FROM 
    Products AS product
INNER JOIN 
    Suppliers AS supplier
    ON product.SupplierID = supplier.SupplierID;

### Exercise 2

The `UNION` operator is used to combine the results of two `SELECT` statements. It removes duplicate rows from the results.

Expected outcome: A single column table of unique cities where customers and employees are located.

In [ ]:
%%sql

SELECT 
    City 
FROM 
    Customers
UNION
SELECT 
    City 
FROM 
    Employees;

###  Exercise 3

The `JOIN` query returns cities that are common in both `Customers` and `Employees` tables (i.e. it looks for matches). On the other hand, the `UNION` query returns a list of all unique cities in both tables, eliminating any duplicates.

Expected outcome: Two sets of outputs – one for the `JOIN` query and another for the `UNION` query.

In [ ]:
%%sql

SELECT 
    Customers.City 
FROM 
    Customers 
INNER JOIN 
    Employees 
    ON Customers.City = Employees.City;


SELECT 
    City 
FROM 
    Customers
UNION
SELECT 
    City 
FROM 
    Employees;

### 4. Challenge question

In [ ]:
%%sql

SELECT 
    customer.ContactName, 
    customer.Country,
    SUM(orderDetails.Quantity * orderDetails.UnitPrice) AS TotalSalesAmount,
    product.ProductName,
    SUM(orderDetails.Quantity) AS TotalUnitsSold
FROM 
    Customers AS customer
JOIN 
    Orders AS orders 
    ON customer.CustomerID = orders.CustomerID
JOIN 
    OrderDetails AS orderDetails 
    ON orders.OrderID = orderDetails.OrderID
JOIN 
    Products AS product 
    ON orderDetails.ProductID = product.ProductID
WHERE 
    orders.OrderDate BETWEEN '1996-08-01' AND '1998-01-30'
GROUP BY 
    customer.ContactName,
    customer.Country, 
    product.ProductName
HAVING 
    SUM(orderDetails.Quantity * orderDetails.UnitPrice) > 5000;


This problem involves pulling data from multiple tables (`Customers`, `Orders`, `OrderDetails`, and `Products`) to create a detailed report. It's testing your ability to use `JOINS`, aggregate functions, grouping, and the `HAVING` clause.

Here's how to break down the problem:

1. **Identify the tables you need to pull data from**: For this problem, you'll need data from the `Customers`, `Orders`, `OrderDetails`, and `Products` tables.
2. **Join these tables**: Use SQL joins to connect these tables. The `Orders` table is connected to `Customers` via `CustomerID`. The `OrderDetails` table is connected to `Orders` via `OrderID` and to `Products` via `ProductID`.
3. **Filter the data**: Use the `WHERE` clause to filter orders to only include those from the last quarter ('2023-04-01' to '2023-06-30').
4. **Aggregate the data**: Aggregate functions are needed to calculate the total amount spent by each customer (SUM of `Quantity` * `UnitPrice`) and the total quantity of each product purchased by each customer (`SUM` of `Quantity`). This is done in the `SELECT` clause.
5. **Group the data**: Use the `GROUP BY` clause to segment the data by `CustomerName`, `Country`, and `ProductName`.
6. **Apply a condition on an aggregate:** Lastly, use the `HAVING` clause to include only customers who have spent more than 5000 in the last quarter. Remember, when you want to use a condition on an aggregate, you should use the `HAVING` clause, not the `WHERE` clause. The `WHERE` clause is used to filter rows, while the `HAVING` clause is used to filter groups.

The result is a detailed report with customer names, their countries, the total amount spent by each customer, the list of products each customer purchased, and the total quantity of each product purchased by each customer for the last quarter. Only customers who spent more than 5000 are included.